# Brute force solver for a 2x2 Rubik's cube


## Naive 2x2 Brute Force Solver

We use MeepMoops's Py222 implementation:

In [52]:
from py222 import py222
import numpy as np

We begin by getting a solved puzzle

In [53]:
s = py222.initState()
print(py222.isSolved(s))
py222.printCube(s)

True
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


Next, we can apply a sequence of moves to scramble the puzzle:

In [54]:
scramble = py222.doAlgStr(s, "x y R U' R'")
# scramble = py222.doAlgStr(s, "x y R U' R' U' F2 U' R")
# scramble = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U")
# scramble = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U R' U F2")
print(py222.isSolved(scramble))
py222.printCube(scramble)

False
      ┌──┬──┐
      │ 1│ 2│
      ├──┼──┤
      │ 2│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 2│ 4│ 3│ 1│ 5│ 0│ 4│ 0│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 3│ 3│ 1│ 2│ 1│ 0│ 4│ 4│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 5│ 3│
      ├──┼──┤
      │ 5│ 5│
      └──┴──┘


Next, we brute force a solution.
Note that we can reduce the degrees of freedom to only nine types of moves: 
* Rotations do not affect state, and can be ignored.
* WLOG, moving the left side is equivalent to moving the right side. Thus, instead of 6 turns ((R)ight, (L)eft, (U)p, (D)own, (F)ront, (B)ack), we can use 3 types of turns ((R)ight, (F)ront, and (U)p).
* For each of the three types of turns, we can rotate clockwise ninety degrees, rotate counterclockwise (indicated by a prime (')), or rotate 180 degrees (indicated by a '2')

Note that God's number for a 2x2 Rubik's cube is 11; i.e., any 2x2 puzzle can be solved within 11 moves. This means, using a naive approach, we can bound our run time as follows:
* Our first move has nine possible options
* Subsequent moves have six possible options (WLOG, if we turned the right face on the _n-1_-th move, we won't want to turn right again in any form on the _n_-th move).

Thus, an upper bound for the number of iterations is O(9*6^10) or 544 million iterations for our naive approach.

We can represent the possible moves:

In [55]:
valid_moves = ["R", "R'", "R2", "U", "U'", "U2", "F", "F'", "F2"]

In [56]:
def solve_222(cube):
    cubes_queue = [(cube, [])]

    num_moves = 0
    i = 0

    while(True):
        # grab the next cube on the queue
        (cube, prev_moves) = cubes_queue.pop(0)

        if py222.isSolved(cube):
            print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            return [valid_moves[move] for move in prev_moves]

        if len(prev_moves) > num_moves:
            print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            num_moves = len(prev_moves)

        prev_move = prev_moves[-1] if len(prev_moves) > 0 else 9

        # Enforce not repeating the same side again
        next_moves = list(range(9))[0:(prev_move//3 * 3)] + list(range(9))[(prev_move//3 * 3 + 3):]

        # Perform each possible move, and add each of those cubes to the queue
        for move in next_moves:
            next_cube = py222.doAlgStr(cube, valid_moves[move])
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

This is very slow for any more than a couple of moves, so we can try a different method of hashing the cubes we've already tried.

(Unfortunately, this upcoming approach is even slower!... likely due to the str(cube) operations.)

In [57]:
def solve_222_bad(cube):
    cubes_already_tried = set([str(cube)])
    cubes_queue = [(cube, [])]

    num_moves = 0

    while(True):
        # grab the next cube on the queue
        (cube, prev_moves) = cubes_queue.pop(0)

        if py222.isSolved(cube):
            return prev_moves

        if len(prev_moves) > num_moves:
            print("Num moves: " + str(len(prev_moves)))
            num_moves = len(prev_moves)

        # Perform each possible move, and add each of those cubes to the queue
        for move in valid_moves:
            next_cube = py222.doAlgStr(cube, move)
            if str(next_cube) not in cubes_already_tried:
                cubes_queue.append((next_cube, prev_moves + [move]))

In [58]:
solve_222(scramble)

Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 10
Num moves: 3	 Iter: 64
Num moves: 3	 Iter: 65


['R', 'U', "R'"]

We now have a naive way to brute force 2x2 solutions. Note that this scales as O(6^n), where n is the number of moves.

## Two-Phase Approach

Next, we try the 2x2 version of Kociemba's algorithm. This involves solving the cube into a "G1" state, where the corners are all oriented. Then, we brute force using a subset of moves: R2, F2, U, U', U2

Recognizing corner orientation is a matter of normalizing* the cube, and then seeing that the top and bottom are all either 0's or 3's.

Normalizing means that we recolor the stickers such that we could have gotten to the state using a single corner we hold fixed.*

In [59]:
def corners_oriented(normalized_cube):
    # This follows the similar idea as the isSolved() method:
    # We check the normalized cube has 0's on the top
    # and 3's on the bottom
    for i in [0, 3]:
        for e in normalized_cube[4 * i:4 * i + 4]:
            if e not in [0, 3]:
                return False
    return True

    a = [0, 1, 2, 3]
    

In [60]:
scr_norm = py222.normFC(scramble)
py222.printCube(scr_norm)
corners_oriented(scr_norm)

      ┌──┬──┐
      │ 2│ 0│
      ├──┼──┤
      │ 0│ 1│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 0│ 5│ 4│ 2│ 3│ 1│ 5│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 0│ 2│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 4│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


False

In [61]:
test_corners_oriented = py222.normFC(py222.doAlgStr(s, "x R U' R F2 R' U R"))
py222.printCube(test_corners_oriented)
print(corners_oriented(test_corners_oriented))

      ┌──┬──┐
      │ 0│ 3│
      ├──┼──┤
      │ 0│ 3│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 5│ 4│ 2│ 2│ 4│ 1│ 2│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 1│ 5│ 4│ 5│ 1│ 2│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 0│
      ├──┼──┤
      │ 3│ 0│
      └──┴──┘
True


Now, we can brute force using 2 steps: we first brute force into a corner oriented state, and then brute force a solution with a more limited move set.

In [62]:
G1_valid_moves = ["R2", "F2", "U", "U'", "U2"]

def next_valid_g1_moves(prev_moves):
    if len(prev_moves) == 0:
        return G1_valid_moves

    prev_move = prev_moves[-1]
    if "U" in prev_move:
        return G1_valid_moves[0:2]
    elif prev_move == "R2":
        return ["F2"] + G1_valid_moves[2:]
    else:
        return ["R2"] + G1_valid_moves[2:]
    
def next_valid_moves(prev_moves):
    prev_move = prev_moves[-1] if len(prev_moves) > 0 else "_"

    # Enforce not repeating the same side again
    return list(filter(lambda move: prev_move[0] not in move, valid_moves))

In [63]:
def solve_to_state(cube, is_in_end_state, next_valid_moves):
    cubes_queue = [(cube, [])]

    num_moves = 0
    i = 0

    while(True):
        # grab the next cube on the queue
        (cube, prev_moves) = cubes_queue.pop(0)

        if is_in_end_state(cube):
            # print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            return [move for move in prev_moves]

        if len(prev_moves) > num_moves:
            # print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            num_moves = len(prev_moves)

        # Perform each possible move, and add each of those cubes to the queue
        for move in next_valid_moves(prev_moves):
            next_cube = py222.doAlgStr(cube, move)
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

def solve_222_multiphase(cube):
    print("Scrambled cube:")
    py222.printCube(cube)
    norm_cube = py222.normFC(cube)
    moves_to_g1 = solve_to_state(norm_cube, corners_oriented, next_valid_moves)
    g1_cube = py222.doAlgStr(norm_cube, " ".join(moves_to_g1))
    print("Cube after phase 1:")
    py222.printCube(g1_cube)
    moves_to_solve = solve_to_state(g1_cube, py222.isSolved, next_valid_g1_moves)
    print("Solved cube:")
    py222.printCube(py222.doAlgStr(norm_cube, " ".join(moves_to_g1 + moves_to_solve)))
    return moves_to_g1 + moves_to_solve

In [64]:
scramble1 = py222.doAlgStr(s, "x y R U' R'")
scramble2 = py222.doAlgStr(s, "x y R U' R' U' F2 U' R")
scramble3 = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U")
scramble4 = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U R' U F2")

In [65]:
solve_222_multiphase(scramble1)

Scrambled cube:
      ┌──┬──┐
      │ 1│ 2│
      ├──┼──┤
      │ 2│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 2│ 4│ 3│ 1│ 5│ 0│ 4│ 0│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 3│ 3│ 1│ 2│ 1│ 0│ 4│ 4│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 5│ 3│
      ├──┼──┤
      │ 5│ 5│
      └──┴──┘
Cube after phase 1:
      ┌──┬──┐
      │ 0│ 3│
      ├──┼──┤
      │ 0│ 3│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 5│ 1│ 1│ 2│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 5│ 1│ 1│ 2│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 0│
      ├──┼──┤
      │ 3│ 0│
      └──┴──┘
Solved cube:
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


['R', 'U', 'R', 'R2']

In [66]:
solve_222_multiphase(scramble2)

Scrambled cube:
      ┌──┬──┐
      │ 0│ 1│
      ├──┼──┤
      │ 2│ 2│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 5│ 0│ 4│ 1│ 0│ 2│ 3│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 3│ 3│ 4│ 5│ 0│ 1│ 5│ 4│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 2│ 4│
      ├──┼──┤
      │ 5│ 3│
      └──┴──┘
Cube after phase 1:
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 3│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 5│ 2│ 1│ 5│ 1│ 4│ 2│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 5│ 4│ 2│ 2│ 1│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 0│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘
Solved cube:
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


['U', 'R', 'U', 'R2', "U'", 'R2', "U'", 'F2', 'U', 'F2', "U'", 'F2', 'U']

In [67]:
solve_222_multiphase(scramble3)

Scrambled cube:
      ┌──┬──┐
      │ 2│ 0│
      ├──┼──┤
      │ 2│ 1│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 1│ 0│ 2│ 3│ 1│ 5│ 0│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 3│ 3│ 4│ 5│ 0│ 1│ 5│ 4│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 2│ 4│
      ├──┼──┤
      │ 5│ 3│
      └──┴──┘
Cube after phase 1:
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 3│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 5│ 2│ 1│ 5│ 1│ 4│ 2│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 5│ 4│ 2│ 2│ 1│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 0│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘
Solved cube:
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


['R', 'U', 'R2', "U'", 'R2', "U'", 'F2', 'U', 'F2', "U'", 'F2', 'U']

In [68]:
solve_222_multiphase(scramble4)

Scrambled cube:
      ┌──┬──┐
      │ 2│ 2│
      ├──┼──┤
      │ 2│ 2│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 0│ 3│ 1│ 4│ 3│ 0│ 4│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 3│ 3│ 1│ 1│ 0│ 0│ 4│ 4│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 5│ 5│
      ├──┼──┤
      │ 5│ 5│
      └──┴──┘
Cube after phase 1:
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 1│ 4│ 2│ 5│ 4│ 1│ 5│ 2│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘
Solved cube:
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


['R2', 'U2', 'F2', 'U', 'R2', 'U2', 'F2', 'U', 'F2', 'U2', 'R2', 'U']

### Update to try multiple Corner Orientation states

In [69]:
def solve_to_state_many(cube, is_in_end_state, next_valid_moves, max_moves, n=1, debug=False):
    cubes_queue = [(cube, [])]
    sols_found = []

    num_moves = 0
    i = 0

    while(True):
        # grab the next cube on the queue
        (cube, prev_moves) = cubes_queue.pop(0)

        if is_in_end_state(cube):
            sols_found.append([move for move in prev_moves])

        if len(prev_moves) > num_moves:
            if debug:
                print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            num_moves = len(prev_moves)

            # exit if we've gone too far in the tree
            if num_moves > max_moves:
                if debug:
                    print("Reached maximum moves: " + str(max_moves))
                return sols_found
            
            # exit if we've now surpassed the number of solutions we're trying to find
            if len(sols_found) >= n:
                if debug:
                    print("Reached maximum num solutions: " + str(len(sols_found)))
                return sols_found

        # Perform each possible move, and add each of those cubes to the queue
        for move in next_valid_moves(prev_moves):
            next_cube = py222.doAlgStr(cube, move)
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

def solve_222_multiphase2(cube, debug=False):
    print("Solving the following cube:")
    py222.printCube(cube)
    norm_cube = py222.normFC(cube)
    
    print("Finding CO candidates")
    moves_to_g1 = solve_to_state_many(norm_cube, corners_oriented, next_valid_moves, 6, n=9001, debug=debug)
    best_move_count_so_far = 1000
    sols_found = []

    print("Searching through " + str(len(moves_to_g1)) + " candidates")
    for i in range(len(moves_to_g1)):
        g1_moves = moves_to_g1[i]
        g1_cube = py222.doAlgStr(norm_cube, " ".join(g1_moves))

        if debug:
            print(g1_moves)

        if best_move_count_so_far - len(g1_moves) >= 0:
            moves_to_solve = solve_to_state_many(g1_cube, py222.isSolved, next_valid_g1_moves, best_move_count_so_far - len(g1_moves), debug=debug)

            j = 0
            for fin_moves in moves_to_solve:
                j = j+1
                if debug:
                    print("Candidate " + str(j) + ":")

                if len(g1_moves + fin_moves) < best_move_count_so_far:
                    best_move_count_so_far = len(g1_moves + fin_moves)
                    sols_found = []
                sols_found.append(g1_moves + fin_moves)
                
                if debug:
                    print(g1_moves + fin_moves)
    
    return sols_found

In [70]:
solve_222_multiphase2(scramble1)

Solving the following cube:
      ┌──┬──┐
      │ 1│ 2│
      ├──┼──┤
      │ 2│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 2│ 4│ 3│ 1│ 5│ 0│ 4│ 0│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 3│ 3│ 1│ 2│ 1│ 0│ 4│ 4│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 5│ 3│
      ├──┼──┤
      │ 5│ 5│
      └──┴──┘
Finding CO candidates
Searching through 246 candidates


[['R', 'U', "R'"]]

In [71]:
solve_222_multiphase2(py222.doAlgStr(py222.initState(), "R U2 R2 F2 R' F2 R F R"), debug=False)

Solving the following cube:
      ┌──┬──┐
      │ 2│ 3│
      ├──┼──┤
      │ 1│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 1│ 3│ 5│ 4│ 2│ 2│ 4│ 3│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 2│ 0│ 0│ 4│ 1│ 0│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 1│ 5│
      ├──┼──┤
      │ 3│ 5│
      └──┴──┘
Finding CO candidates
Searching through 88 candidates


[["U'", 'F', 'R', 'F2', 'R', "U'", 'R2', "U'", 'F2', 'R2', "U'"],
 ["U'", 'F', 'R', 'F2', 'R', "U'", 'F2', 'R2', 'U', 'F2', "U'"],
 ["U'", "R'", 'U', 'F', "U'", 'F', 'U2', 'F2', 'U', 'R2', 'U2'],
 ["U'", "R'", 'U', 'F', "U'", "F'", 'U2', 'F2', "U'", 'R2', 'U2'],
 ["U'", 'F', 'R', 'F2', 'R', "U'", 'R2', "U'", 'F2', 'R2', "U'"],
 ["U'", 'F', 'R', 'F2', 'R', "U'", 'F2', 'R2', 'U', 'F2', "U'"],
 ['F', 'U', "F'", 'U2', "R'", "F'", "U'", 'F2', "U'", 'R2', 'U2']]

We see that trying multiple corner orientation candidates does have a longer runtime, but does output better solutions.

## Pruning tables

We now explore pruning tables, which attempt to store the number of moves required to solve from a given set of states. This lets us recover a solution in linear time, since the movecounts act as breadcrumbs that let us recover solutions for all states stored in the table.

We generate the number of moves every single "G1" state takes:

In [72]:
powers_of_2 = [1, 2, 4, 8]
powers_of_6 = [6**i for i in range(4)]


def hash_g1(cube):
    up_hash = np.dot(cube[0:4], powers_of_2)
    down_hash = np.dot(cube[4 * 3:4 * 3 + 4], powers_of_2)

    front_hash = np.dot(cube[4 * 2:4 * 2 + 4], powers_of_6)
    back_hash = np.dot(cube[4 * 5:4 * 5 + 4], powers_of_6)
    fb_hash = front_hash * (6**4) + back_hash

    return fb_hash * (2**8) + up_hash * (2**4) + down_hash
            

In [73]:
def calc_g1_movecount():
    g1_movecount = dict()
    q = [(py222.initState(), [])]

    while(len(q) > 0):
        (cube, prev_moves) = q.pop(0)

        cube_hash = hash_g1(cube)

        if cube_hash not in g1_movecount:
            g1_movecount[cube_hash] = len(prev_moves)
            
            for move in G1_valid_moves:
                next_cube = py222.doAlgStr(cube, move)
                q.append((next_cube, prev_moves + [move]))
    
    return g1_movecount

g1_movecount = calc_g1_movecount()

In [74]:
len(g1_movecount)

5040

Since there are so few G1 states (only 7!), we could just store all the solutions directly, as opposed to breadcrumbs:

In [75]:
def calc_g1_sols():
    g1_sols = dict()
    q = [(py222.initState(), [])]

    while(len(q) > 0):
        (cube, prev_moves) = q.pop(0)

        cube_hash = hash_g1(cube)

        if cube_hash not in g1_sols:
            g1_sols[cube_hash] = prev_moves
            
            for move in G1_valid_moves:
                next_cube = py222.doAlgStr(cube, move)
                q.append((next_cube, prev_moves + [move]))
    
    return g1_sols

g1_sols = calc_g1_sols()

def lookup_g1_sol(cube):
    tree_sol = g1_sols[hash_g1(cube)]
    return (len(tree_sol), tree_sol[::-1])

In [76]:
test_cube = py222.doAlgStr(py222.initState(), "R2 U F2 U2 R R' U'")
lookup_g1_sol(test_cube)

(4, ['U', 'F2', 'U', 'R2'])

We now have a new multiphase algorithm that uses this pruning table:

In [77]:
def solve_222_multiphase3(cube, debug=False):
    print("Solving the following cube:")
    py222.printCube(cube)
    norm_cube = py222.normFC(cube)
    
    print("Finding CO candidates")
    moves_to_g1 = solve_to_state_many(norm_cube, corners_oriented, next_valid_moves, 5, n=9001, debug=debug)
    best_move_count_so_far = 1000
    sol_found = None

    print("Searching through " + str(len(moves_to_g1)) + " candidates")
    for i in range(len(moves_to_g1)):
        g1_moves = moves_to_g1[i]
        g1_cube = py222.doAlgStr(norm_cube, " ".join(g1_moves))

        if debug:
            print(g1_moves)

        if best_move_count_so_far - len(g1_moves) >= 0:
            (fin_move_count, fin_moves) = lookup_g1_sol(g1_cube)

            if len(g1_moves) + fin_move_count < best_move_count_so_far:
                best_move_count_so_far = len(g1_moves) + fin_move_count
                sol_found = g1_moves + fin_moves
            
            if debug:
                print(g1_moves + fin_moves)
    
    return sol_found

Finally, we get a 2x2x2 that, while far from optimal, generates solutions very quickly:

In [78]:
solve_222_multiphase3(scramble4)

Solving the following cube:
      ┌──┬──┐
      │ 2│ 2│
      ├──┼──┤
      │ 2│ 2│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 0│ 3│ 1│ 4│ 3│ 0│ 4│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 3│ 3│ 1│ 1│ 0│ 0│ 4│ 4│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 5│ 5│
      ├──┼──┤
      │ 5│ 5│
      └──┴──┘
Finding CO candidates
Searching through 600 candidates


['U', 'R2', 'U2', 'F2', 'U', 'F2', 'U2', 'R2', 'U', 'F2', 'U2', 'R2']

In [79]:
solve_222_multiphase3(py222.doAlgStr(py222.initState(), "R U2 R2 F2 R' F2 R F R"), debug=False)

Solving the following cube:
      ┌──┬──┐
      │ 2│ 3│
      ├──┼──┤
      │ 1│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 1│ 3│ 5│ 4│ 2│ 2│ 4│ 3│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 2│ 0│ 0│ 4│ 1│ 0│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 1│ 5│
      ├──┼──┤
      │ 3│ 5│
      └──┴──┘
Finding CO candidates
Searching through 10 candidates


["U'", 'F', 'R', 'F2', 'R', 'U', 'R2', 'U', 'F2', 'R2', 'U']